# Funkcje:

In [3]:
### WYMIANA KLUCZA DIFFIEGO-HELLMANA
def Diffie_Hellman_key_exchange(p,g,a,b):
    
    if is_prime(p) == True:
    
        if Mod(g,p).is_primitive_root() == True:
            
            x = power_mod(g,a,p)
            y = power_mod(g,b,p)
            s_k_a = power_mod(y,a,p)
            s_k_b = power_mod(x,b,p)
            s_k_a_b = power_mod(g,a*b,p)

            print("Użytkownicy uzgodnili liczbę pierwszą p = ",p)
            print("")
            print("Za jej pierwiastek pierwotny wzięli liczbę g = ",g)
            print("")
            print("Użytkownik A wybrał tajną liczbę :",a,", oraz przesłał użytkownikowi B wartość :",x)
            print("")
            print("Użytkownik B wybrał tajną liczbę :",b,", oraz przesłał użytkownikowi A wartość :",y)
            print("")
            print("Wspólnym tajnym kluczem obliczonym przez A jest :",s_k_a)
            print("")
            print("Wspólnym tajnym kluczem obliczonym przez B jest :",s_k_b)
            print("")
            print("Sprawdzamy zgodność kluczy :",s_k_a_b)
            
        else:
            print("Uzgodniona liczba g nie jest pierwiastkiem pierwotnym.")
    else:
        print("Wybrana liczba p nie jest pierwsza.")

### ALGORYTM GENEROWANIA KLUCZA ELGAMAL        
def ElGamal_public_key(p,g,k):
    
    if is_prime(p) == True:
        
        if Mod(g,p).is_primitive_root() == True:

            if k > 0 and k < p-1:
                x = power_mod(g,k,p)
                print("Użytkownik wybrał liczbę pierwszą: p = ",p)
                print("")
                print("Za jej pierwiastek pierwotny wziął liczbę: g = ",g)
                print("")
                print("Jego kluczem prywatnym jest liczba: k = ",k)
                print("")
                print("Klucz publiczny tego użytkownika to trójka:",(p,g,x))
                K = (p,g,x)
                
            else:
                print("Liczba wybrana przez użytkownika jest nieodpowiednia.")
        else:
            print("Uzgodniona liczba g nie jest pierwiastkiem pierwotnym.")
    else:
        print("Wybrana liczba p nie jest pierwsza.")
        
    return K

### FUNKCJE POMOCNICZE
def letters_to_indices(s):
    return [ord(char.upper()) - ord('A') for char in s if char.isalpha()]

def indices_to_letters(indices):
    return ''.join(chr(index + ord('A')) for index in indices)


### ALGORYTM SZYFROWANIA ELGAMAL
def ElGamal_encryption_algorithm(message,users_key,j):
    p = users_key[0]
    g = users_key[1]
    x = users_key[2]
    
    if j > 0 and j < p-1:
        M = letters_to_indices(message)

        C1 = power_mod(g,j,p)
        xj = power_mod(x,j,p)

        cipher = [(M[k]*xj)%p for k in range(len(M))]
        cipher_output = (C1,cipher)
        
        print("Użytkownik chce przesłać wiadomość do osoby o kluczu publicznym :",users_key)
        print("")
        print("Wiadomość którą chce wysłać to :",message)
        print("")
        print("Wybrał liczbę j = ",j)
        print("")
        print("Szyfrogramem jego wiadomośći jest :",cipher_output)
    else:
        print("Wybrana liczba j jest nieodpowiednia.")
    
    return cipher_output


### ALGORYTM DESZYFROWANIA ELGAMAL
def ElGamal_decryption_algorithm(cipher,secret_key,public_key):
    C1 = cipher[0]
    cipher_input = cipher[1]
    k = secret_key
    p = public_key[0]
    
    C1_rec = power_mod(C1,-k,p)
    decripted_cipher = [(cipher_input[k]*C1_rec)%p for k in range(len(cipher_input))]
    
    decrypted_message = indices_to_letters(decripted_cipher)
    
    print("Odszyfrowana wiadomość to:", decrypted_message)
    
    return decrypted_message


### METODA PRZELICZANIA
def Brute_force_method(a,b,n, details = False):
    if details == True:
        print("Rozważamy zagadnienie:",a,"^x =",b,"( mod",n,")\n")
    al = Mod(b,n)
    bl = Mod(a,n)
    i = 0
    y = Mod(1,n)
    while True:
        y = y*bl
        if y ==al:
            break
        i +=1
    if details == True:
        print("Wynik końcowy x =", i+1)
    return i+1

### ALGORYTM MAŁYCH I WIELKICH KROKÓW
def Baby_giant_step(a,b,n,details = False, sets_display=False):
    Zn = Integers(n)
    al = Zn(b)
    bl = Zn(a)
    m = ceil(sqrt(n))
    B = [al*bl^(-r) for r in range(m)]
    delta = bl^Integer(m)
    j = 0
    delq = Zn(1)
    delty = []
    if details == True:
        print("Rozważamy zagadnienie:",a,"^x =",b,"( mod",n,")\n")
        print("m =",m)
        print("")
    while j<m:
        j += 1
        delq = delq * delta
        delty.append(delq)
        for i in range(m):
            if delq==B[i]:
                x = m*j + i
                j=m
                break
    if details == True:
        if sets_display == True:
            print("Zbiór małych kroków:")
            print(B)
            print("")
            print("Zbiór wielkich kroków:")
            print(delty)
            print("")
        print("Pierwszy element zbioru wielkich kroków, który pokrył się z poprzednikiem w zbiorze małych kroków, to:",delq)
        print("")
        print("Ma on numer: j =",len(delty),", oraz odpowiada małemu krokowi z numerem: i =",B.index(delq))
        print("")
        print("Zgodnie ze wzorem, mamy: x =",len(delty),"*",m,"+",B.index(delq))
        print("")
        print("Wynik końcowy: x =",x)
    return x


### ALGORYTM POHLIGA-HELLMANA
def Pohlig_Hellman_algorithm(a,b,n, details = False):
    if details == True:
        print("Zagadnienie:",a,"^x =",b,"( mod",n,"), można zastąpić przez: \n")
    fa = factor(n-1)
    fl = len(fa)
    x=[]
    y=[]
    p_list=[]
    e_list=[]
    for i in range(fl):
        p=fa[i][0]
        e=fa[i][1]
        np = (n-1)/p^e
        bp = power_mod(b,np,n)
        ap = power_mod(a,np,n)
        
        disc = discrete_log(Mod(bp,n),Mod(ap,n))
        p_list.append(p)
        e_list.append(e)
        x.append(disc)
        y.append(fa[i][0]^fa[i][1])
        if details == True:
            print(ap,"^x(",p,") =",bp," (mod",n,") z rozwiązaniem x(",p,") = ", disc) 
    if details == True:
        print("")
        print("Zgodnie z twierdzeniem:")
        for i in range(fl):
            print("x =",x[i]," (mod", p_list[i],"^",e_list[i],")")

        print("\nWynik końcowy: x =",CRT_list(x,y))
    return CRT_list(x,y)

### PORÓWNANIE ALGORYTMÓW
import time
def Compare_algorithms(a,b,n):
    start_time = time.time()
    result = Brute_force_method(a, b, n)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Czas wykonania: Metoda przeliczania:",elapsed_time,"sekund\n")
    
    start_time = time.time()
    result = Baby_giant_step(a, b, n)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Czas wykonania: Algorytm małych i wielkich kroków:",elapsed_time,"sekund\n")
    
    start_time = time.time()
    result = Pohlig_Hellman_algorithm(a, b, n)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Czas wykonania: Algorytm Diffiego-Hellmana:",elapsed_time,"sekund")

# Wymiana klucza Diffiego-Hellmana

In [5]:
p=19
g=2
a=7
b=4
Diffie_Hellman_key_exchange(p,g,a,b)


Użytkownicy uzgodnili liczbę pierwszą p =  19

Za jej pierwiastek pierwotny wzięli liczbę g =  2

Użytkownik A wybrał tajną liczbę : 7 , oraz przesłał użytkownikowi B wartość : 14

Użytkownik B wybrał tajną liczbę : 4 , oraz przesłał użytkownikowi A wartość : 16

Wspólnym tajnym kluczem obliczonym przez A jest : 17

Wspólnym tajnym kluczem obliczonym przez B jest : 17

Sprawdzamy zgodność kluczy : 17


# Generowanie klucza w systemie ElGamal'a

In [6]:
p=113
g=3
k=37
ElGamal_public_key(p,g,k)

Użytkownik wybrał liczbę pierwszą: p =  113

Za jej pierwiastek pierwotny wziął liczbę: g =  3

Jego kluczem prywatnym jest liczba: k =  37

Klucz publiczny tego użytkownika to trójka: (113, 3, 24)


(113, 3, 24)

# Algorytm szyfrowania ElGamal

In [8]:
message="SZYFR"
users_key=(43,3,22)
j=23
ElGamal_encryption_algorithm(message,users_key,j)

Użytkownik chce przesłać wiadomość do osoby o kluczu publicznym : (43, 3, 22)

Wiadomość którą chce wysłać to : SZYFR

Wybrał liczbę j =  23

Szyfrogramem jego wiadomośći jest : (34, [17, 26, 37, 31, 28])


(34, [17, 26, 37, 31, 28])

# Algorytm deszyfrowania ElGamal

In [12]:
C1=34
C2="[17, 26, 37, 31, 28]"
secret_key=15
public_key="(43,3,22)"

cipher = (C1,eval(C2))
public_key = eval(public_key)
ElGamal_decryption_algorithm(cipher,secret_key,public_key)

Odszyfrowana wiadomość to: SZYFR


'SZYFR'

# Metoda przeliczania

In [13]:
a=2
b=7
n=557
Brute_force_method(a,b,n, details = True)

Rozważamy zagadnienie: 2 ^x = 7 ( mod 557 )

Wynik końcowy x = 458


458

# Alogrytm małych i wielkich kroków

In [14]:
a=2
b=7
n=557
details=True
sets_display=True
Baby_giant_step(a, b, n, details, sets_display)

Rozważamy zagadnienie: 2 ^x = 7 ( mod 557 )

m = 24

Zbiór małych kroków:
[7, 282, 141, 349, 453, 505, 531, 544, 272, 136, 68, 34, 17, 287, 422, 211, 384, 192, 96, 48, 24, 12, 6, 3]

Zbiór wielkich kroków:
[376, 455, 81, 378, 93, 434, 540, 292, 63, 294, 258, 90, 420, 289, 49, 43, 15, 70, 141]

Pierwszy element zbioru wielkich kroków, który pokrył się z poprzednikiem w zbiorze małych kroków, to: 141

Ma on numer: j = 19 , oraz odpowiada małemu krokowi z numerem: i = 2

Zgodnie ze wzorem, mamy: x = 19 * 24 + 2

Wynik końcowy: x = 458


458

# Alogrytm Pohliga-Hellmana

In [15]:
a=2
b=7
n=557
details=True
Pohlig_Hellman_algorithm(a, b, n, details)

Zagadnienie: 2 ^x = 7 ( mod 557 ), można zastąpić przez: 

118 ^x( 2 ) = 556  (mod 557 ) z rozwiązaniem x( 2 ) =  2
16 ^x( 139 ) = 173  (mod 557 ) z rozwiązaniem x( 139 ) =  41

Zgodnie z twierdzeniem:
x = 2  (mod 2 ^ 2 )
x = 41  (mod 139 ^ 1 )

Wynik końcowy: x = 458


458

# Porównanie czasu działania algorytmów

In [19]:
Compare_algorithms(2,7,1344546617)

Czas wykonania metody przeliczania: 147.0667700767517 sekund

Czas wykonania algorytmu małych i wielkich kroków: 33.723758935928345 sekund

Czas wykonania alogrytmu Diffiego-Hellmana: 0.008737564086914062 sekund
